In [2]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 66.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 48.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.9 MB/s eta 0:00:00


In [3]:
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

In [15]:
type(load_dataset)

function

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [39]:
import pandas as pd
from datasets import Dataset

# Specify the correct file path to your CSV file

# file_path = "/kaggle/input/training-hackathon-2023/train_ccdm.csv"

file_path = "/content/drive/MyDrive/text/train_dataset.csv"


df = pd.read_csv(file_path)

# Convert the pandas DataFrame to a datasets Dataset
datasetnew = Dataset.from_pandas(df)

In [40]:
type(datasetnew)

datasets.arrow_dataset.Dataset

In [8]:
with open(r"/content/drive/MyDrive/text/output.txt", 'r') as fp:
    for count, line in enumerate(fp):
        pass
print('Total Number of lines:', count + 1)

Total Number of lines: 9296


In [41]:
# The model that you want to train from the Hugging Face hub
model_name = "NousResearch/Llama-2-7b-chat-hf"

# The instruction dataset to use
dataset_name = datasetnew

# Fine-tuned model name
new_model = "llama-2-7b-miniguanaco"

################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of training epochs
num_train_epochs = 1

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 4

# Batch size per GPU for evaluation
per_device_eval_batch_size = 4

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule
lr_scheduler_type = "cosine"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 0

# Log every X updates steps
logging_steps = 25

################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

In [44]:
print(datasetnew)

Dataset({
    features: ['Text', 'Content'],
    num_rows: 7436
})


In [45]:
def formatting_prompts_func(data):
    output_texts = []

    # Split the data into prompts and answers
    prompts = data["Text"]
    answers = data["Content"]

    for i in range(len(prompts)):
        text = f"""
        <s>[INST]
        {{Question: {prompts[i]}}} [/INST]

        {{Answer: {answers[i]}}} </s>
        """
        output_texts.append(text)
    return output_texts

In [46]:
# Load dataset (you can process it here)

dataset = datasetnew

# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    #dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    formatting_func=formatting_prompts_func,
    packing=packing,
)

# Train model
trainer.train()

# Save trained model
trainer.model.save_pretrained(new_model)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:159: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/7436 [00:00<?, ? examples/s]

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
25,5.001000
50,2.609500
75,1.278000
100,0.465600
125,0.893400
150,0.352800
175,0.764400
200,0.339900
225,0.728100
250,0.319500


In [47]:
model_to_save = trainer.model.module if hasattr(trainer.model, 'module') else trainer.model
model_to_save.save_pretrained("outputs")

In [50]:
import pandas as pd
from datasets import Dataset

# Specify the correct file path to your CSV file

# file_path = "/kaggle/input/training-hackathon-2023/train_ccdm.csv"

file_path = "/content/drive/MyDrive/text/test_dataset.csv"


df2 = pd.read_csv(file_path)

# Convert the pandas DataFrame to a datasets Dataset
dataset_test = Dataset.from_pandas(df2)

In [51]:
type(dataset_test)

datasets.arrow_dataset.Dataset

In [ ]:
def formatting_prompts_func(data):
    output_texts = []

    # Split the data into prompts and answers
    prompts = data["Text"]
    answers = data["Content"]

    for i in range(len(prompts)):
        text = f"""
        <s>[INST]
        {{Question: {prompts[i]}}} [/INST]

        {{Answer: {answers[i]}}} </s>
        """
        output_texts.append(text)
    return output_texts

In [53]:
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
# for i in range(len(test_dataset)):
#   text = f"""
#   <s>[INST]
#   {{Question: {test_dataset[i]}}} [/INST]"""
#   pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
#   result = pipe(f"<s>[INST] {test_dataset[i]} [/INST]")
#   print(result[0]['generated_text'])


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will

<s>[INST] t [/INST]  I'm not sure I understand what you are saying. Unterscheidung? Can you explain?
<s>[INST] e [/INST]  I'm not sure I understand what you are saying. Unterscheidung between "e" and "i" is not possible in the context of this conversation. Could you please provide more information or clarify your question?
<s>[INST] s [/INST]  I'm not sure I understand what you are saying. Unterscheidung? Can you please provide more context or clarify your question?
<s>[INST] t [/INST]  I'm not sure I understand what you are saying. Unterscheidung between "t" and "t"? Could you explain?


In [54]:
from peft import LoraConfig, get_peft_model

In [55]:
lora_config = LoraConfig.from_pretrained('outputs')
model = get_peft_model(model, lora_config)

In [56]:
device = "cuda:0"
model = model.to(device)

text = f"""
<s>[INST]
{{Question: <s>[INST] declared by the customer.\\nActual Conversation:\'\'\'\\nCustomer: Why green tea is not better then blue tea\'\'\'\\nInformation}} [/INST]

{{Answer:
"""

# Tokenize and generate on the same device
inputs = tokenizer(text, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=512)
# Decode the generated response
response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(response)



<s>[INST] 
{Question:[INST] declared by the customer.\nActual Conversation:'''\nCustomer: Why green tea is not better then blue tea'''\nInformation} [/INST]

{Answer:
Green tea and blue tea are both popular types of tea that have their own unique characteristics and health benefits. While both are made from the Camellia sinensis plant, they are processed differently, which affects their flavor and nutritional content.

Green tea is made by steaming or pan-frying the leaves, which helps preserve the tea's natural antioxidants and catechins. Green tea has been shown to have numerous health benefits, including improving heart health, aiding in weight loss, and reducing the risk of certain diseases such as cancer and Alzheimer's.

Blue tea, on the other hand, is made by fermenting the tea leaves, which gives it a distinctive blue-gray color and a more complex flavor profile. Blue tea has been shown to have antibacterial and anti-inflammatory properties, and may also have potential health 